<a href="https://colab.research.google.com/github/Alton01/ML-breast-cancer-prediction-with-pytorch/blob/main/breast_cancer_prediction_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
#load breast cancer dataset
data = load_breast_cancer()
x, y = data.data, data.target

In [4]:
#dataset splitting into training and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
print(x.shape)
print(x_train.shape)
print(x_test.shape)
print(y.shape)
print(y_train.shape)
print(y_test.shape)

(569, 30)
torch.Size([455, 30])
torch.Size([114, 30])
(569,)
torch.Size([455])
torch.Size([114])


In [6]:
# standardize the data using standard scaler. This ensures the data is normally distributed. mean is going to be (0) zero and S.D is 1
#logistic regression expects the data to be normally distributed.
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [7]:
type(x_train)

numpy.ndarray

In [8]:
# convert data from numpy.ndarray to pytorch tensors as we cannot use numpy.ndarray in pytorch.
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

In [9]:
# Neural Network Architecture
# input_size represents how many number of neurons should be present in input layer
# input layer contains the same number of neurons as the features
# hidden_size represents how many number of neurons should be present in hidden layer
# output_size represents how many number of neurons should be present in output layer
# outpt_size depends on how many classes the classification is into. in this binary classification, it is 2 neureons.
# ReLU (Rectified Linear Unit) activation function. This non-linear function is typically applied after the first fully connected layer
# to introduce non-linearity into the model, allowing it to learn more complex patterns.
#  Sigmoid activation function is often used in the output layer of binary classification models to squash the output values between 0 and 1.

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) #1st fully connected layer
        self.relu = nn.ReLU() #(Rectified Linear Unit) activation function
        self.fc2 = nn.Linear(hidden_size, output_size) #2nd fully connected layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [10]:
# Define hyperparameters
input_size = x_train.shape[1] # Number of features in the input data which is 30
hidden_size = 64
output_size = 1  ## a single neuron that would give the value as either 0 or 1. output of logistic regression
learning_rate = 0.001
num_epochs = 100

In [11]:
# initialize the neural network and move it to the GPU

model = NeuralNet(input_size, hidden_size, output_size).to(device)


In [18]:
# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
#Training the neural network model

for epoch in range(num_epochs):
   model.train()
   optimizer.zero_grad() #zero_grad resets the gradient to zero
   outputs = model(x_train)
   loss = criterion(outputs, y_train.view(-1,1))
   loss.backward()
   optimizer.step()

   #calculate accuracy
   with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1, 1)).float().sum() #counting how many predictions (are true) are the same as the y_train and summing it.
    accuracy = correct / y_train.size(0)

   if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%')


Epoch [10/100], Loss: 0.5994, Accuracy: 66.59%
Epoch [20/100], Loss: 0.4710, Accuracy: 89.67%
Epoch [30/100], Loss: 0.3702, Accuracy: 92.75%
Epoch [40/100], Loss: 0.2939, Accuracy: 93.19%
Epoch [50/100], Loss: 0.2386, Accuracy: 94.51%
Epoch [60/100], Loss: 0.1992, Accuracy: 94.95%
Epoch [70/100], Loss: 0.1710, Accuracy: 95.82%
Epoch [80/100], Loss: 0.1502, Accuracy: 96.04%
Epoch [90/100], Loss: 0.1345, Accuracy: 96.70%
Epoch [100/100], Loss: 0.1223, Accuracy: 97.14%


In [21]:
#model evaluation on training data
model.eval()
with torch.no_grad():
  outputs = model(x_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1, 1)).float().sum()
  accuracy = correct / y_train.size(0)
  print(f'Training Accuracy: {accuracy.item() * 100:.2f}%')

Training Accuracy: 97.14%


In [22]:
#model evaluation on test data
model.eval()
with torch.no_grad():
  outputs = model(x_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1, 1)).float().sum()
  accuracy = correct / y_test.size(0)
  print(f'Test Accuracy: {accuracy.item() * 100:.2f}%')

Test Accuracy: 98.25%
